In [ ]:
import pywt
import matplotlib.pyplot as plt
from numpy.fft import fft
import numpy as np
import librosa

In [ ]:
SAMPLE_RATE = 1600

"""
saw, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\waves\\saw.wav", sr=SAMPLE_RATE)
sine, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\waves\\sine.wav", sr=SAMPLE_RATE)
square, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\waves\\square.wav", sr=SAMPLE_RATE)
up, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\waves\\filterUp.wav", sr=SAMPLE_RATE)
down, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\waves\\filterDown.wav", sr=SAMPLE_RATE)
flute, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\waves\\basic flute.wav", sr=SAMPLE_RATE)
"""

signal, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\validationdata\\53.wav", sr = SAMPLE_RATE)

In [ ]:
def plotWaveform(ax, signal, sr, numpoints):
    librosa.display.waveshow(signal, sr=sr, ax=ax, max_points=numpoints)
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Amplitude")

def plotSpectrogram(ax, signal, sr):
	ax.specgram(signal, sr, cmap="viridis")

def plotScaleogram(ax, signal, waveletname="cmor"):
	scales = np.arange(1, 200)  
	coeffs, _ = pywt.cwt(signal, scales, waveletname)
	ax.imshow(np.abs(coeffs), extent=[0, len(signal), 1, 200], cmap="viridis", aspect="auto", origin="lower")
	ax.xlabel("Sample")
	ax.ylabel("Scale")

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=3)

signals = [sine, saw, square]
for i in range(0,3):
	plotWaveform(axes[0, i], signals[i], SAMPLE_RATE, int(3 * SAMPLE_RATE / 261))
	plotSpectrogram(axes[1,i], signals[i], SAMPLE_RATE)
	plotScaleogram(axes[2,i], signals[i])

fig.show()

#Plotting code

In [ ]:
#todo replace spectrogram images with this
stft = librosa.stft(down)
db = librosa.amplitude_to_db(np.abs(stft), ref=np.max)
plt.figure()
librosa.display.specshow(S_db)

In [ ]:
plt.specgram(flute, SAMPLE_RATE, cmap="viridis", Fs=SAMPLE_RATE)
plt.axis(ymax = 1000)
plt.axis("off")
plt.show()

In [ ]:
#show a scaleogram
signal, _ = librosa.load("C:\\Users\\abdulg\\source\\repos\\Synth\\backwards\\validationdata\\420.wav", sr = SAMPLE_RATE)
scales = np.arange(1, 30)  
coeffs, _ = pywt.cwt(signal, scales, "mexh")
plt.imshow(np.abs(coeffs), extent=[0, len(signal), 1, 30], cmap="viridis", aspect="auto", origin="lower")
plt.axis("off")
plt.show()

In [ ]:
#fft plotting code
signal = flute
spectrum = np.fft.fft(signal)
frequencies = np.fft.fftfreq(len(signal), d=1/16000)

# Plot only positive frequencies and their magnitudes
plt.plot(frequencies[:len(frequencies)//10], np.abs(spectrum)[:len(frequencies)//10])
plt.xlabel('Frequency (Hz)')
plt.ylabel('Magnitude')

plt.show()

In [ ]:
#too big!
np.shape(coeffs)

In [ ]:
#try downsampling, but we introduce 'ripples' in the image that could confuse the CNN for an LFO
coeffs2 = block_reduce(coeffs, (2,32), np.max)
print(np.shape(coeffs2))
plt.imshow(np.abs(coeffs2), extent=[0, len(signal), 1, 80], cmap="viridis", aspect="auto", origin="lower")
plt.axis("off")
plt.show()

In [ ]:
coeffs2 = dct(coeffs, type=2, axis=1)

In [ ]:
EXAMPLES_PER_BATCH = 32
WAVELET_NAME = "mhat"
WAVELET_SCALES = np.arange(1, 30)  
SAMPLE_RATE = 16000
SAMPLES_PER_EXAMPLE = 64000
NUM_WAVES = 3
NUM_OTHER_FEATURES = 9
NUM_DCT_COEFFICIENTS = 1024
from keras.models import Model
from keras.layers import Input, Conv2D, Dense, MaxPooling2D,Flatten
inputLayer = Input(shape=(len(WAVELET_SCALES), NUM_DCT_COEFFICIENTS, 1))
conv1 = Conv2D(16, (2,3), strides=(1,2), activation="relu")(inputLayer)
pool1 = MaxPooling2D(pool_size=(2, 2), strides=(2,2), padding="valid")(conv1)
conv2 = Conv2D(32, (2,2), activation="relu")(pool1)
pool2 = MaxPooling2D(pool_size=(3, 3), strides=(3,3), padding="valid")(conv2)
conv3 = Conv2D(64, (2,2), activation="relu")(pool2)
#flatten and then bifurcate the network into the classification and regression parts
flat = Flatten()(conv3)

In [ ]:
print(flat.shape)

In [ ]:
matrix = np.array([
	[1,2,3,4,5,6, 100],
	[7,8,9,10,11,12, 200],
	[13,14,15,16,17,18, 300]
])
np.split(matrix, [3], axis=1)

In [ ]:
np.split(matrix, [3], axis=1)

In [ ]:
import pickle
with open("./lastHistory", "rb") as histFile:
	history = pickle.load(histFile)
	plt.plot(history["loss"])
	plt.plot(history["val_loss"])
	plt.title("model loss")
	plt.ylabel("loss")
	plt.xlabel("epoch")
	plt.legend(["train", "val"], loc="upper left")
	plt.show()